In [ ]:
import pandas as pd
import numpy as np
import requests
from google.colab import data_table
from matplotlib import pyplot
from scipy.stats import pearsonr
import plotly.express as px

data_table.enable_dataframe_formatter()

FPL General Information API Calling

In [ ]:
fpl_general_info_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

fpl_general_info_r = requests.get(fpl_general_info_url)

fpl_general_info_json = fpl_general_info_r.json()

fpl_general_info_json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

Creating a Teams Code Mapping DF

In [ ]:
teams_df = pd.DataFrame(fpl_general_info_json['teams'])

teams_df

teams_code_mapping = teams_df[['id','name']]

teams_code_mapping = teams_code_mapping.rename(columns={"name":"team","id":"team_id"})

# teams_code_mapping

Now creating a Player Position Mapping

In [ ]:
element_types_df = pd.DataFrame(fpl_general_info_json['element_types'])

fpl_position_points_df = element_types_df[['id','singular_name_short']]

fpl_position_points_df = fpl_position_points_df.rename(columns={"id":"pos_id","singular_name_short":"pos"})

elements_df = pd.DataFrame(fpl_general_info_json['elements'])

elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [ ]:
player_details_df = elements_df[['element_type','id','first_name','second_name','web_name','team']]
player_details_df = player_details_df.rename(columns={'element_type':'position_id',
                                            'id':'player_id',
                                            'team':'team_code'
                                            })
player_details_df['fpl_cleansheet_position_points'] = np.where(player_details_df['position_id'] == 1,4,
                                                               np.where(player_details_df['position_id'] == 2,4,
                                                                        np.where(player_details_df['position_id'] == 3,1,0)))

player_details_df['fpl_goals_position_points'] = np.where(player_details_df['position_id'] == 1,6,
                                                               np.where(player_details_df['position_id'] == 2,6,
                                                                        np.where(player_details_df['position_id'] == 3,5,4)))

# player_details_df

Now loading individual players season history for game-by-game df.

Fixtures is future current season.
History is previous current season.
History Past is previous seasons

In [ ]:
element_summary_id = 1

player_stats_game_by_game_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(element_summary_id)+'/'
player_stats_game_by_game_r = requests.get(player_stats_game_by_game_url)
player_stats_game_by_game_json = player_stats_game_by_game_r.json()
player_game_stats_loop = pd.DataFrame(player_stats_game_by_game_json['history'])

In [ ]:
element_summary_id = 1

for element_summary_id in range(1,752):
  element_summary_id +1
  player_stats_game_by_game_player_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(element_summary_id)+'/'
  player_stats_game_by_game_player_r = requests.get(player_stats_game_by_game_player_url)
  player_stats_game_by_game_player_json = player_stats_game_by_game_player_r.json()
  player_stats_game_by_game_player_json.keys()
  player_game_stats_player = pd.DataFrame(player_stats_game_by_game_player_json['history'])
  player_game_stats_loop = player_game_stats_loop.append(player_game_stats_player, ignore_index = True)

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-70-f81619719f4c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [ ]:
player_game_stats = player_game_stats_loop.drop_duplicates()

In [ ]:
player_game_stats.columns

Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out'],
      dtype='object')

Cleaning Player_Game_Stats

In [ ]:
player_game_stats = player_game_stats_loop.drop_duplicates()

player_game_stats = player_game_stats[['element','fixture','opponent_team','total_points','was_home','round','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps','starts','value','expected_goals', 'expected_assists','expected_goals_conceded']]

player_game_stats = player_game_stats.rename(columns={"fixture":"fixture_id",
                                                      "opponent_team":"opponent_team_id",
                                                      "element":"player_code",
                                                      "total_points":"round_points"})

player_game_stats = player_game_stats.merge(teams_code_mapping, how='left',left_on ='opponent_team_id',right_on='team_id')

player_game_stats = player_game_stats.rename(columns={"team":"opposition_team"})

player_game_stats = player_game_stats[['player_code', 'fixture_id', 'opponent_team_id', 'round_points',
       'was_home', 'round', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'starts', 'value', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'opposition_team']]

player_game_stats = player_game_stats.merge(player_details_df,how='left',left_on='player_code',right_on='player_id')

player_game_stats = player_game_stats.merge(teams_code_mapping, how='left',left_on ='team_code',right_on='team_id')

player_game_stats['position'] = np.where(player_game_stats['position_id'] == 1,'GK',
                                         np.where(player_game_stats['position_id'] == 2,'DEF',
                                                  np.where(player_game_stats['position_id'] == 3,'MID',
                                                           np.where(player_game_stats['position_id'] == 4,'FWD','Fail'))))

player_game_stats['value'] = player_game_stats['value']*100000

player_game_stats['expected_goals'] = pd.to_numeric(player_game_stats['expected_goals'])
player_game_stats['expected_goals'] = round(player_game_stats['expected_goals'],2)
player_game_stats['expected_assists'] = pd.to_numeric(player_game_stats['expected_assists'])
player_game_stats['expected_goals_conceded'] = pd.to_numeric(player_game_stats['expected_goals_conceded'])

player_game_stats['home_or_away'] = np.where(player_game_stats['was_home'] == True,'H','A')

player_game_stats['fixture'] = player_game_stats['opposition_team']+' ('+player_game_stats['home_or_away']+')'

# player_game_stats = player_game_stats[['player_code','first_name', 'second_name', 'web_name','fixture','value','round','round_points','starts','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
#        'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
#        'red_cards', 'saves'
#        , 'bonus'
#        ,'expected_goals', 'expected_assists','expected_goals_conceded']]

player_game_stats.columns

Index(['player_code', 'fixture_id', 'opponent_team_id', 'round_points',
       'was_home', 'round', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'starts', 'value', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'opposition_team', 'position_id',
       'player_id', 'first_name', 'second_name', 'web_name', 'team_code',
       'fpl_cleansheet_position_points', 'fpl_goals_position_points',
       'team_id', 'team', 'position', 'home_or_away', 'fixture'],
      dtype='object')

Creating Expected FPL Points (GK|DEF|MID|FWD)
* Expected Goals (6|6|5|4)
* Expected Assists (3|3|3|3)
* Expected Cleansheets (4|4|1|0)
* Expected Goals Conceded Per 2 goals (-1|-1|0|0)
* Minutes are the same
* Penalties Saved are zero
* Penalties Missed are zero
* Yellow cards are the same
* Red Cards are the same
* Own Goals are 0
* Saves are the same - until I can think of a method
* Bonus is the same - until I can think of a method


In [ ]:
# Goals
# player_game_stats['expected_fpl_points_goals'] = player_game_stats['expected_goals']*4
player_game_stats['expected_fpl_points_goals'] = player_game_stats['expected_goals']*player_game_stats['fpl_goals_position_points']
# Assists
player_game_stats['expected_fpl_points_assists'] = player_game_stats['expected_assists']*3
# Minutes
player_game_stats['fpl_points_minutes'] = np.where(player_game_stats['minutes'] > 60,2,
                                                    np.where(player_game_stats['minutes'] > 0,1,0))
# # Goals Conceded
player_game_stats['expected_goals_conceded_rounded'] = round(player_game_stats['expected_goals_conceded'])
player_game_stats['expected_fpl_points_goals_conceded'] = np.where(player_game_stats['position'] == 'GK',np.floor(player_game_stats['expected_goals_conceded_rounded']/2),
                                                                   np.where(player_game_stats['position'] == 'DEF',np.floor(player_game_stats['expected_goals_conceded_rounded']/2),0))

# # Cleansheets
player_game_stats['expected_fpl_cleansheet_eligible'] = np.logical_and(player_game_stats['expected_goals_conceded_rounded'] == 0,
                                                                       player_game_stats['minutes'] > 60)

player_game_stats['expected_fpl_points_cleansheet'] = np.where(player_game_stats['expected_fpl_cleansheet_eligible'] == True, player_game_stats['fpl_cleansheet_position_points'],0)
                                                                                                                               


# EXPECTED FPL POINTS
player_game_stats['expected_fpl_points'] = round((player_game_stats['expected_fpl_points_goals']
                                             +player_game_stats['expected_fpl_points_assists']
                                             +player_game_stats['fpl_points_minutes']
                                             +(np.floor(player_game_stats['saves']/3))
                                             +player_game_stats['expected_fpl_points_cleansheet']
                                             +player_game_stats['bonus']
                                             -player_game_stats['expected_fpl_points_goals_conceded']
                                             -(player_game_stats['yellow_cards'])
                                             -(player_game_stats['red_cards']*3)
                                             ),2)

In [ ]:
# player_game_stats

In [ ]:
player_game_stats.dtypes

player_code                             int64
fixture_id                              int64
opponent_team_id                        int64
round_points                            int64
was_home                                 bool
round                                   int64
minutes                                 int64
goals_scored                            int64
assists                                 int64
clean_sheets                            int64
goals_conceded                          int64
own_goals                               int64
penalties_saved                         int64
penalties_missed                        int64
yellow_cards                            int64
red_cards                               int64
saves                                   int64
bonus                                   int64
bps                                     int64
starts                                  int64
value                                   int64
expected_goals                    

In [ ]:
player_game_stats_cleaned = player_game_stats[['player_code','first_name','web_name','second_name','position_id','position','team','fixture','value','round','round_points','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves'
       ,'bonus',
       'expected_goals','expected_assists','expected_goals_conceded'
       ,'expected_fpl_points']]

player_game_stats_cleaned

,player_code,first_name,web_name,second_name,position_id,position,team,fixture,value,round,...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,expected_goals,expected_assists,expected_goals_conceded,expected_fpl_points
0,1,Cédric,Cédric,Alves Soares,2,DEF,Fulham,Crystal Palace (A),4500000,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,1,Cédric,Cédric,Alves Soares,2,DEF,Fulham,Leicester (H),4400000,2,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,1,Cédric,Cédric,Alves Soares,2,DEF,Fulham,Bournemouth (A),4400000,3,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,1,Cédric,Cédric,Alves Soares,2,DEF,Fulham,Fulham (H),4300000,4,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,1,Cédric,Cédric,Alves Soares,2,DEF,Fulham,Aston Villa (H),4300000,5,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18115,748,Romaine,Mundle,Mundle,3,MID,Spurs,Southampton (A),4500000,28,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
18116,749,George,G.Shelvey,Shelvey,1,GK,Nott'm Forest,Spurs (A),4000000,27,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
18117,749,George,G.Shelvey,Shelvey,1,GK,Nott'm Forest,Newcastle (H),4000000,28,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
18118,750,Charlie,C.Robinson,Robinson,2,DEF,Fulham,Arsenal (H),4000000,27,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


Cleaning Data for Grouping (sum), so fixtures and value needs additional cleaning

In [ ]:
player_round_stats_added_info = player_game_stats_cleaned.groupby(['round','player_code']).agg({'fixture':' / '.join}).reset_index()

player_round_stats_added_info = player_round_stats_added_info.rename(columns={'fixture':'fixture(s)'})

player_round_stats_added_info_value = player_game_stats[['round','player_code','value']].groupby(['round','player_code']).max('value').reset_index()

# player_round_stats_added_info_value

player_round_stats_added_info = player_round_stats_added_info.merge(player_round_stats_added_info_value,how='left',left_on=['round','player_code'],right_on=['round','player_code'])

# player_round_stats_added_info

In [ ]:
player_round_stats = player_game_stats_cleaned[['player_code','first_name','web_name','second_name','position_id','position','team','round','round_points','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves'
       ,'bonus','expected_goals','expected_assists','expected_goals_conceded'
       ,'expected_fpl_points']]

# player_round_stats['game_count'] = '1'

# player_round_stats

player_round_stats = player_round_stats.groupby(['player_code','first_name','web_name','second_name','position_id','position','team','round']).sum().reset_index()

player_round_stats['expected_fpl_points'] = round(player_round_stats['expected_fpl_points'],2)

player_round_stats = player_round_stats.merge(player_round_stats_added_info,how='left',left_on=['round','player_code'],right_on=['round','player_code'])

# player_round_stats.columns

player_round_stats = player_round_stats[['player_code','first_name','web_name','second_name','position_id','position','team','round','value','fixture(s)','round_points','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves'
       ,'bonus','expected_goals','expected_assists','expected_goals_conceded'
       ,'expected_fpl_points']]

player_round_stats = player_round_stats.sort_values(by=['round_points','expected_fpl_points','value'],ascending=[False,False,True])

player_round_stats

,player_code,first_name,web_name,second_name,position_id,position,team,round,value,fixture(s),...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,expected_goals,expected_assists,expected_goals_conceded,expected_fpl_points
489,19,Gabriel,Martinelli,Martinelli Silva,3,MID,Arsenal,25,6500000,Leicester (A) / Everton (H),...,0,0,1,0,0,6,1.06,0.30,1.38,16.20
7318,280,Virgil,Van Dijk,van Dijk,2,DEF,Liverpool,25,6500000,Crystal Palace (A) / Wolves (H),...,0,0,0,0,0,6,0.67,0.02,1.53,14.08
8257,318,Erling,Haaland,Haaland,4,FWD,Man City,9,12100000,Man Utd (H),...,0,0,0,0,0,3,1.11,0.76,1.67,11.72
7274,279,Roberto,Firmino,Firmino,4,FWD,Liverpool,4,8000000,Bournemouth (H),...,0,0,0,0,0,3,1.18,0.43,0.33,11.01
7394,283,Mohamed,Salah,Salah,3,MID,Liverpool,26,12700000,Man Utd (H),...,0,0,1,0,0,3,0.57,0.50,0.88,8.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12564,482,Nélson,Semedo,Cabral Semedo,2,DEF,Wolves,15,5000000,Brighton (H),...,0,0,0,1,0,0,0.00,0.00,2.39,-3.00
9731,376,Nick,Pope,Pope,1,GK,Newcastle,24,5500000,Liverpool (H),...,0,0,0,1,0,0,0.00,0.00,3.49,-3.00
3968,153,James,Tomkins,Tomkins,2,DEF,Crystal Palace,17,3900000,Fulham (H),...,0,0,0,1,0,0,0.00,0.00,4.41,-4.00
4408,170,Tyrick,Mitchell,Mitchell,2,DEF,Crystal Palace,17,4500000,Fulham (H),...,0,0,0,1,0,0,0.00,0.00,4.51,-4.00


Building Basic Graphs to compare data...

Need to have graph df, then make the following:
* Expected vs Actual
* Round Performance for each player

In [ ]:
graph_df = player_round_stats.sort_values(by=['position_id','round_points','expected_fpl_points','value'],ascending=[True,False,False,True])

graph_df['player_name'] = graph_df['first_name']+' '+graph_df['second_name']

graph_df['goals_icon'] = np.where(graph_df['goals_scored'] == 0,None,graph_df['round_points']+1)

graph_df['assists_icon'] = np.where(graph_df['assists'] == 0,None,
                                    np.where(graph_df['goals_scored'] == 0,graph_df['round_points']+1,
                                             graph_df['goals_icon']+1))

Scatter for Expected vs Actual

In [ ]:
expected_vs_actual = px.scatter(graph_df,x='round_points',y='expected_fpl_points',hover_name = 'player_name',
                 hover_data = {'position':False,
                              #  'round_point':False,
                              #  'expected_fpl_points':False,
                               'web_name':True,
                               'team':True,
                               'fixture(s)':True},
                 color = "position",
                 symbol = "position",
                 facet_col = "position",
                 trendline = "ols")

expected_vs_actual.update_layout(title = "Expected vs Actual FPL Points per Round by Position")

expected_vs_actual.show()

Line for Player Performance over Round

In [ ]:
graph_df.dtypes

saka = graph_df.loc[(graph_df.player_code == 13)]

saka.columns

# 'player_code','first_name','web_name','second_name','position_id','position','team','round','value','fixture(s)','round_points','minutes','goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
#        'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
#        'red_cards', 'saves'
#        ,'bonus','expected_goals','expected_assists','expected_goals_conceded'
#        ,'expected_fpl_points'

Index(['player_code', 'first_name', 'web_name', 'second_name', 'position_id',
       'position', 'team', 'round', 'value', 'fixture(s)', 'round_points',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'expected_goals', 'expected_assists',
       'expected_goals_conceded', 'expected_fpl_points', 'player_name',
       'goals_icon', 'assists_icon'],
      dtype='object')

In [ ]:
fig = px.line(saka.sort_values('round'),x='round',y='expected_fpl_points',markers = True,
              labels = {'expected_fpl_points':'Expected FPL Points',
                        'round':'Round',
                        'round_points':'Actual Points',
                        'fixture(s)':'Fixture(s)',
                            'minutes':'Minutes',
                            'goals_scored':'Goals Scored',
                            'expected_goals':'Expected Goals',
                            'assists':'Assists',
                            'expected_assists':'Expected Assists',
                            'goals_conceded':'Goals Conceded',
                            'clean_sheets':'Clean Sheet(s)',
                            'expected_goals_conceded':'Expected Goals Conceded'},
              hover_name = 'player_name',
              hover_data = ['round_points',
                            'fixture(s)',
                            'minutes',
                            'goals_scored',
                            'expected_goals',
                            'assists',
                            'expected_assists',
                            'goals_conceded',
                            'clean_sheets',
                            'expected_goals_conceded'])
fig.add_bar(x=saka['round'], y=saka['round_points'], name="Actual Points")
fig.add_scatter(x=saka['round'], y=saka['goals_icon'],mode = 'markers',marker=dict(
    size = 15,
    color = 'black'
))
fig.add_scatter(x=saka['round'], y=saka['assists_icon'],mode = 'markers',marker=dict(
    size = 15,
    color = 'red',
    # name = 'Ass'
))
fig.update_layout(template = 'simple_white')

fig.show()

Build dream team options... 

* No Rules (no budget, unlimited from one team)
* FPL (100mil, max 3 per team)
* Expected Points


In [ ]:
# dream_team_info = player_round_stats[['round','first_name','web_name','second_name','team','position','value','fixture(s)','round_points','expected_fpl_points']]

# dream_team_info = dream_team_info.sort_values(by=['round_points','expected_fpl_points','value'],ascending=[False,False,True])

# dream_team_info['fpl_round_rank'] = round(dream_team_info.groupby('round')['round_points'].rank(method='first',ascending=False),0)

# dream_team_info['fpl_round_pos_rank'] = round(dream_team_info.groupby(['position','round'])['round_points'].rank(method='first',ascending=False),0)

# dream_team_info['dt_fpl_no_rules_squad'] = np.where(np.logical_and(dream_team_info['fpl_round_pos_rank'] <= 2, dream_team_info['position'] == 'GK'),True,
#                                           np.where(np.logical_and(dream_team_info['fpl_round_pos_rank'] <= 5, dream_team_info['position'] == 'DEF'),True,
#                                                    np.where(np.logical_and(dream_team_info['fpl_round_pos_rank'] <= 5, dream_team_info['position'] == 'MID'),True,
#                                                             np.where(np.logical_and(dream_team_info['fpl_round_pos_rank'] <= 3, dream_team_info['position'] == 'FWD'),True,False))))

# dream_team_info = dream_team_info.sort_values(by=['expected_fpl_points','round_points','value'],ascending=[False,False,True])

# dream_team_info['xfpl_round_rank'] = round(dream_team_info.groupby('round')['expected_fpl_points'].rank(method='first',ascending=False),0)

# dream_team_info['xfpl_round_pos_rank'] = round(dream_team_info.groupby(['position','round'])['expected_fpl_points'].rank(method='first',ascending=False),0)

# dream_team_info['dt_xfpl_no_rules_squad'] = np.where(np.logical_and(dream_team_info['xfpl_round_pos_rank'] <= 2, dream_team_info['position'] == 'GK'),True,
#                                           np.where(np.logical_and(dream_team_info['xfpl_round_pos_rank'] <= 5, dream_team_info['position'] == 'DEF'),True,
#                                                    np.where(np.logical_and(dream_team_info['xfpl_round_pos_rank'] <= 5, dream_team_info['position'] == 'MID'),True,
#                                                             np.where(np.logical_and(dream_team_info['xfpl_round_pos_rank'] <= 3, dream_team_info['position'] == 'FWD'),True,False))))

# dream_team_info

In [ ]:
# dream_team_info_squad_round = dream_team_info

# dream_team_info_squad_round_grouped = dream_team_info_squad_round[['round','position','round_points','expected_fpl_points']].groupby(['round','position']).sum().reset_index()

# dream_team_info_squad_round_grouped = dream_team_info_squad_round_grouped.sort_values(['round_points','expected_fpl_points'],ascending = [False,False])

# dream_team_info_squad_round_grouped['round_points_position_points'] = dream_team_info_squad_round_grouped.groupby(['position'])['round_points'].rank(method='first',ascending=False)

# dream_team_info_squad_round_grouped